## Tuning Hyperparameter for knn ContentBasedFiltering

Run: 

<i> '$' pipenv shell </i>

<i> '$' pipenv install ipynb </i>

to be able to import functions from other ipynb files


In [2]:
import pandas as pd
import numpy as np
import os
import sys
from knn_preprocessing import knn_preprocessing
#Import will run notebook and perform PCA
from ipynb.fs.full.ContentBasedFiltering import predict_movie_rating, test_predict_mr, get_data

PCA:
Main Variance impacting factors:
[0.1352569  0.09503712 0.07998231 0.06074159 0.04796725 0.04597668
 0.0430597  0.04134133 0.03750373 0.03703565 0.03502601 0.0336219
 0.03295991 0.03155028 0.0294954  0.02847374 0.02764069 0.02351684
 0.02326137 0.01913027 0.01828914 0.01731806 0.0156969  0.01428033
 0.01294528 0.00832974 0.0045619 ]
     weights        features  abs_weights
2   0.460198      imdbRating     0.460198
5   0.428491      Metacritic     0.428491
4   0.422208  RottenTomatoes     0.422208
6   0.352119          Awards     0.352119
3   0.305617       imdbVotes     0.305617
1   0.277951         Runtime     0.277951
14  0.209256               7     0.209256
0  -0.148325            Year     0.148325
11 -0.143027               4     0.143027
25  0.120398              18     0.120398
17 -0.086768              10     0.086768
12  0.083021               5     0.083021
16  0.067442               9     0.067442
18  0.049997              11     0.049997
22 -0.047021              15  

In [1]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [4]:
def kreuzvalidiere(n, folds, mean = False, knn_metric = 'cosine', set_k=False, k_neighbors=15, data=get_data()):
    upper = 0
    lower = 10
    
    doku = pd.DataFrame(columns={'RMSE','Testsize','Fold','mean', 'knn_metric','set_k','k_neighbors','data'})
    
    
    for i in range(0,folds):
        rmse = test_predict_mr(n, mean, knn_metric, set_k, k_neighbors)
        doku.loc[i,'RMSE'] = rmse
        doku.loc[i,'Testsize'] = n
        doku.loc[i,'Fold'] = str(i)
        doku.loc[i,'mean'] = mean
        doku.loc[i,'knn_metric'] = knn_metric
        doku.loc[i,'set_k'] = set_k
        doku.loc[i,'data'] = str(data.columns)
        if set_k:
            doku['k_neighbors'] = k_neighbors
        
        
        if (float(rmse) < float(lower)):
            lower = rmse

        if (float(rmse) > float(upper)):
            upper = rmse

    print('RMSE upper Bound: '+str(upper))
    print('RMSE lower Bound: '+str(lower))
    
    #build output csv
    doku = doku.sort_values(by=['RMSE'], ascending=True)
    doku = doku.dropna(how='all', axis=1)
    doku.to_csv('../../data/tuning/ContentbasedTuning_'+str(round(lower,4))+'.csv') 

## Tuning for Mean() vs. distance-weighted rating

In [32]:
n = 100
folds = 4
print("Testing for n= "+str(n)+", Mean: False"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, mean=False)

Testing for n= 100, Mean: False, Folds: 4
RMSE: 0.804577548248386
RMSE: 0.8327401654270914
RMSE: 0.7110886702295112
RMSE: 0.8678981722445336
RMSE upper Bound: 0.8678981722445336
RMSE lower Bound: 0.7110886702295112


In [10]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", Mean: False"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, mean=False)

Testing for n= 25000, Mean: False, Folds: 10
RMSE: 0.8093717866581247
RMSE: 0.8109298741225405
RMSE: 0.8122008042821607
RMSE: 0.804284010080862
RMSE: 0.8091955357611241
RMSE: 0.8068427998071319
RMSE: 0.8090052843861771
RMSE: 0.8122322697998022
RMSE: 0.8059915436817127
RMSE: 0.8087654925187531
RMSE upper Bound: 0.8122322697998022
RMSE lower Bound: 0.804284010080862


In [11]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", Mean: True"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, mean=True)

Testing for n= 25000, Mean: True, Folds: 10
RMSE: 0.8104320862629049
RMSE: 0.812571557942916
RMSE: 0.8105755766394918
RMSE: 0.8158718554745932
RMSE: 0.8194891221568065
RMSE: 0.8190834153694807
RMSE: 0.8160188609612186
RMSE: 0.8127711506527119
RMSE: 0.8084195815842717
RMSE: 0.8103183042621105
RMSE upper Bound: 0.8194891221568065
RMSE lower Bound: 0.8084195815842717


## Tuning for knn_metric

In [12]:
n = 100
folds = 4
print("Testing for n= "+str(n)+", Metric: 'cosine'"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, knn_metric = 'cosine')

Testing for n= 100, Metric: 'cosine', Folds: 4
RMSE: 0.7201151679152322
RMSE: 0.8245112956227939
RMSE: 0.8076094224428513
RMSE: 0.8226267898123077
RMSE upper Bound: 0.8245112956227939
RMSE lower Bound: 0.7201151679152322


In [ ]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", Metric: 'cosine'"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, knn_metric = 'cosine')

Testing for n= 25000, Metric: 'cosine', Folds: 10
RMSE: 0.810439104119757
RMSE: 0.808101040622642
RMSE: 0.8070554252186012
RMSE: 0.808497482150073
RMSE: 0.8004682317644266
RMSE: 0.8075338510177409
RMSE: 0.8156765064661976
RMSE: 0.8083017533747673
RMSE: 0.8102161937950325


In [5]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", Metric: 'minkowski'"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, knn_metric = 'minkowski')

Testing for n= 25000, Metric: 'minkowski', Folds: 10
RMSE: 0.8154954452823058
RMSE: 0.8078088347765844
RMSE: 0.8095793815054434
RMSE: 0.8102205771189291
RMSE: 0.8148882262054463
RMSE: 0.815599683801152
RMSE: 0.8127600153623162
RMSE: 0.8139401770627214
RMSE: 0.8137384421833501
RMSE: 0.8064772089398172
RMSE upper Bound: 0.815599683801152
RMSE lower Bound: 0.8064772089398172


FileNotFoundError: [Errno 2] No such file or directory: '../../data/tuning/ContentbasedTuning_0.8065.csv'

## Tuning for k_neighbors

In [ ]:
n = 100
folds = 4
print("Testing for n= "+str(n)+", set_k: False"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, set_k=False)

In [ ]:
n = 25000
folds = 10
print("Testing for n= "+str(n)+", set_k: False"+", Folds: "+str(folds))
kreuzvalidiere(n, folds, set_k=False)

In [ ]:
n = 25000
folds = 10
for i in range(4,10)
    print("Testing for n= "+str(n)+", set_k: True,"+"k= "+str(i)+ "Folds: "+str(folds))
    kreuzvalidiere(n, folds, set_k=True, k_neighbors=i)

In [ ]:
n = 25000
folds = 10
for i in range(10,20)
    print("Testing for n= "+str(n)+", set_k: True,"+"k= "+str(i)+ "Folds: "+str(folds))
    kreuzvalidiere(n, folds, set_k=True, k_neighbors=i)

In [ ]:
n = 25000
folds = 10
for i in range(20,30)
    print("Testing for n= "+str(n)+", set_k: True,"+"k= "+str(i)+ "Folds: "+str(folds))
    kreuzvalidiere(n, folds, set_k=True, k_neighbors=i)

## Tuning for input features

In [ ]:
data = get_data()

In [ ]:
data

In [ ]:
#TODO Functionen umschreiben so, dass man auch auf feature Hyperparameter tunen kann 
#->kreuzvalidiere und test_predict_mr müssen 'data' parameter bekommen

In [ ]:
n = 25000
folds = 10
data = data['Zu testende Feature Spalten eintragen']
print("Testing for n= "+str(n)+", Features: Tbd,"+ "Folds: "+str(folds))
kreuzvalidiere(n, folds, set_k=True, k_neighbors=i, data)